In [41]:
import pandas as pd
import seaborn as sns
import numpy as np

df1 = pd.read_csv('2019_2021_TRD_Mnth.csv')
df2 = pd.read_csv('MBK_SHIBORM_New.csv')
df3 = pd.read_csv('2019_2021_IDX_Idxtrdmth.csv')

In [26]:
# Then we change the 'Markettype' to the index of corresponding market
df1.drop(index=(df1.loc[(df1['Markettype']==32)].index), inplace=True)

df1['Markettype'].replace(1,2, inplace=True)
df1['Markettype'].replace(4,399107, inplace=True)
df1['Markettype'].replace(16,399006, inplace=True)

# Delete missing data

df1.dropna(axis=0, how='any')

,Stkcd,Trdmnt,Mretwd,Mretnd,Markettype
0,1,2019-07,0.025399,0.025399,399107
1,1,2019-08,0.002123,0.002123,399107
2,1,2019-09,0.100989,0.100989,399107
3,1,2019-10,0.042976,0.042976,399107
4,1,2019-11,-0.059656,-0.059656,399107
...,...,...,...,...,...
91197,605500,2021-02,0.007151,0.007151,2
91198,605500,2021-03,0.074823,0.074823,2
91199,605500,2021-04,-0.051829,-0.051829,2
91200,605500,2021-05,0.001608,0.001608,2


In [13]:
df2 ## Shibor Rate

,Unnamed: 0,SgnDate,Term,Shibor,ShiborM
0,19087,2019-07,1天,2.642,0.220
1,19260,2019-08,1天,2.572,0.214
2,19422,2019-09,1天,2.635,0.220
3,19568,2019-10,1天,2.590,0.220
4,19741,2019-11,1天,2.356,0.196
5,19906,2019-12,1天,1.694,0.141
6,20044,2020-01,1天,2.444,0.204
7,20210,2020-02,1天,1.628,0.136
8,20385,2020-03,1天,1.614,0.135
9,20556,2020-04,1天,1.888,0.157


In [5]:
df3

,Indexcd,Month,Idxrtn
0,2,2019-07,-0.015467
1,2,2019-08,-0.015634
2,2,2019-09,0.006496
3,2,2019-10,0.008334
4,2,2019-11,-0.019319
...,...,...,...
67,399107,2021-02,-0.017784
68,399107,2021-03,-0.033264
69,399107,2021-04,0.036709
70,399107,2021-05,0.052620


In [6]:
# Detects missing data (Delete)
stk_table = df1.groupby(['Stkcd'])['Stkcd'].agg('count')
# for key, num in stk_table:
stk_table

Stkcd
1         24
2         24
4         24
5         24
6         24
          ..
605398     6
605399    11
605488     2
605499     2
605500     7
Name: Stkcd, Length: 4108, dtype: int64

In [30]:
a = df1['Stkcd'][0]
count = 0
index = -1
list1 = []  # to store the return rate of the company
list2 = []  # to store the shibor rate
list3 = []  # to store the index return
## To store the regressors
listX = []
listy = []

beta_list = []

for i in range(len(df1)):
    index += 1
    if df1['Stkcd'][index] == a:
        count += 1
        list1.append(df1['Mretwd'][i])
        for j in range(len(df2)): # Done!
            if df1['Trdmnt'][i] == df2['SgnDate'][j]:
                list2.append(df2['ShiborM'][j])
        for k in range(len(df3)): 
            if df1['Trdmnt'][i] == df3['Month'][k]:
                if df1['Markettype'][i] == df3['Indexcd'][k]:
                    list3.append(df3['Idxrtn'][k])
    else:
        # if count == 1:
        #     df1.drop(index=(df1['Stkcd']==a).index)
        #     continue
        Sxx = Sxy = 0
        # Derive X and y
        for m in range(list2.__len__()):
            listX.append(list3[m]-list2[m])
            listy.append(list1[m]-list2[m])
        for a in range(listX.__len__()):
            Sxy += listy[a]*(listX[a]-sum(listX)/len(listX))
            Sxx += (listX[a]-sum(listX)/len(listX))*(listX[a]-sum(listX)/len(listX))
        try:
            beta = Sxy/Sxx
            beta_list.append(beta)
        except:
            #! Needs to be fixed (index=693)
            # df1.drop(index=(df1['Stkcd']==693).index)
            df1.drop(index=(df1['Stkcd']==a).index)
            print(a)
            
        ## print(beta_list)
        # Move to the next company
        a = df1['Stkcd'][index]
        # Initialize the list
        count = 0
        list1 = []
        list2 = []
        list3 = []
        listX = []
        listy = []
        
# beta_list

<ipython-input-30-187f13c58d55>:38: RuntimeWarning: invalid value encountered in double_scalars
  beta = Sxy/Sxx


300984
300996
300997
300998
301002
301003
301004
301005
301006
301007
301008
301009
301010
301011
301012
301013
301015
301016
301019
600401
600905
601156
601528
601665
603171
603529
605011
605259
605319


In [68]:
from pandas.core.frame import DataFrame
c = {'beta': beta_list}
beta_df = DataFrame(c)
beta_df.dropna(axis=0, how='any', inplace=True)
beta_df.describe()
# beta_df.to_excel('betalist.xlsx')

sns.distplot(beta_df, rug=False, hist=True)
sns.kdeplot(beta_list, shade=False)
beta_df.plot(kind='kde', xlim=(0,3))

,beta
count,4040.000000
mean,0.962134
std,5.498724
min,-148.497089
25%,0.604012
50%,0.873025
75%,1.149896
max,255.256152
